In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [10]:

# Initialize data and variables
data = []
page_number = 1
max_pages = 1
prix_min = 1
# Define the base URL to prepend to relative links
base_url = "https://www.bestbuy.com/en-ca"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

links = []

while page_number <= max_pages:
    url = f"https://www.bestbuy.com/site/promo/laptop-and-computer-deals?cp={page_number}"
    
    response = requests.get(url, headers=headers)
    print(response.text)
    soup = BeautifulSoup(response.text, "html.parser")

    divs = soup.find_all("div", class_="shop-sku-list-item")
    print(divs)
    if len(divs) == 0:
        break

    print(f"Page-{page_number} ", end=' ')

    for div in divs:

        column_left = div.find("div", class_="column-left")
        if column_left:
            # Find the <a> tag inside column-left
            anchor = column_left.find("a")
            if anchor and anchor.has_attr("href"):
                clink = base_url + anchor["href"]
                links.append(clink)
                print(f"Found link: {clink}", end=' ')

    print()
    page_number += 1

print(links)

<!doctype html>
<html class="no-js" lang="en">
<head>
  <meta charset="utf-8">
  <meta http-equiv="x-ua-compatible" content="ie=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta http-equiv="content-type" content="text/html" />
  <meta name="keywords" content=" best buy international, best buy countries" />
  <meta name="description" content="Shop online at Best Buy in your country and language of choice. Best Buy provides online shopping in a number of countries and languages."
  />
  <link rel="icon" href="https://www.bestbuy.com/~assets/bby/_com/header-footer/images/favicon-32.png" sizes="32x32">
  <title>Best Buy International: Select your Country - Best Buy</title>
  <link rel="stylesheet" href="https://www.bestbuy.com/~assets/bby/_intl/landing_page/css/app.css">
</head>
<body>
  <div class="language-menu">
    <ul class="container">
      <li>
        <button lang="en">English</button>
      </li>
      <li>
        <button lang="fr">FranÃ§ais

In [6]:

scraped_data = []

for i, link in enumerate(links):

    print(f"link-{i+1}", end=' ')
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    try:
        # Name
        name_tag = soup.find("h2", itemprop="name")
        name = name_tag.get_text(strip=True) if name_tag else None

        # Phone (in span.hide > a[href^=tel:])
        phone_tag = soup.select_one("button.btn-tel span.hide a[href^='tel']")
        phone = phone_tag.get_text(strip=True) if phone_tag else None

        # Address (full text from <p itemprop="streetAddress"> and <strong itemprop="addressLocality">)
        address_p = soup.find("p", itemprop="streetAddress")
        locality_span = soup.find("strong", itemprop="addressLocality")
        address = (
            (address_p.get_text(" ", strip=True) if address_p else "") +
            (" " + locality_span.get_text(strip=True) if locality_span else "")
        ).strip()

        scraped_data.append({
            "name": name,
            "phone": phone,
            "address": address,
            "link" : link
        })

    except Exception as e:
        print(f"Error scraping {link}: {e}")


In [5]:
df_orig = pd.DataFrame(scraped_data)
df = df_orig.copy()
df.head()

""


In [14]:
df_clean = df.dropna(subset=['phone'])

In [15]:
df_clean.to_csv(f'data/entreprises_{city}.csv', index=False)
df_clean.to_excel(f'data/entreprises_{city}.xlsx')

In [10]:
import requests
from bs4 import BeautifulSoup
import json


def scrape_bestbuy_ca_product(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract title
    title_tag = soup.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else "N/A"

    # Extract short description from the meta tag
    description_tag = soup.find("meta", {"name": "description"})
    description = description_tag["content"] if description_tag else "N/A"

    # Extract embedded JSON from <script type="application/ld+json">
    specs = {}
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string.strip())
            if isinstance(data, dict) and data.get("@type") == "Product":
                specs = {
                    "brand": data.get("brand", {}).get("name", ""),
                    "sku": data.get("sku", ""),
                    "name": data.get("name", ""),
                    "category": data.get("category", ""),
                }
        except Exception:
            continue

    return {
        "title": title,
        "description": description,
        "specifications": specs
    }


# Example usage
url = "https://www.bestbuy.ca/en-ca/product/google-pixel-9a-128gb-obsidian-unlocked/18933640"
data = scrape_bestbuy_ca_product(url)
print(json.dumps(data, indent=2))

{
  "title": "Google Pixel 9a 128GB - Obsidian - Unlocked",
  "description": "Create without limits and crush your goals with Google Pixel 9a, AI-powered to help you stay ahead. With durable design, built-in Gemini<sup> 1</sup>, and battery that keeps up with you, Pixel 9a brings your wildest ideas to life all day, every day. Capture stunning shots with its exceptional camera. And seven years of updates to help it feel like new.",
  "specifications": {}
}
